In [2]:
import numpy as np
import cv2 as cv
import glob
import os

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((9*6,3), np.float32)  # 9x6 체스보드에 맞게 수정
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
# 이미지 디렉토리 경로 확인
images_dir = '/Users/yangdongjae/Desktop/2025/DGIST/Computer_Vision/Assignment/step.6/data/'
if not os.path.exists(images_dir):
    print(f"디렉토리가 존재하지 않습니다: {images_dir}")
    exit(1)

# 모든 이미지 형식 시도
image_formats = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tif', '*.heic']
images = []
for fmt in image_formats:
    images.extend(glob.glob(os.path.join(images_dir, fmt)))

print(f"총 {len(images)}개 이미지 파일을 찾았습니다.")
if len(images) == 0:
    print("이미지 파일을 찾을 수 없습니다. 디렉토리 내용:")
    for file in os.listdir(images_dir):
        print(f"- {file}")
    exit(1)

for fname in images:
    print(f"이미지 처리 중: {os.path.basename(fname)}")
    img = cv.imread(fname)
    if img is None:
        print(f"이미지를 읽을 수 없습니다: {fname}")
        
        # 파일 정보 출력
        if os.path.exists(fname):
            print(f"  파일 크기: {os.path.getsize(fname)} 바이트")
            print(f"  파일 확장자: {os.path.splitext(fname)[1]}")
        else:
            print(f"  파일이 존재하지 않습니다.")
        continue
    
    print(f"  이미지 크기: {img.shape}")
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (9,6), None)
 
    # If found, add object points, image points (after refining them)
    if ret:
        print(f"  체스보드 코너 발견!")
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
 
        # Draw and display the corners
        cv.drawChessboardCorners(img, (9,6), corners2, ret)
        # cv.imshow('img', img)
        # cv.waitKey(500)
    else:
        print(f"  체스보드 코너를 찾을 수 없습니다.")
 
cv.destroyAllWindows()

# 캘리브레이션 수행 (코너를 찾은 이미지가 있는 경우)
if len(objpoints) > 0:
    print(f"\n{len(objpoints)}개 이미지에서 캘리브레이션 수행 중...")
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
        objpoints, imgpoints, gray.shape[::-1], None, None
    )
    
    print("\n캘리브레이션 결과:")
    print(f"카메라 행렬:\n{mtx}")
    print(f"\n왜곡 계수:\n{dist}")
    
    # 결과 저장
    np.savez('camera_calibration.npz', 
             camera_matrix=mtx,
             dist_coeffs=dist,
             rvecs=rvecs,
             tvecs=tvecs)
    print("\n캘리브레이션 데이터가 'camera_calibration.npz'에 저장되었습니다.")
    
    # K 행렬을 txt 파일로 저장
    np.savetxt('camera_matrix_K.txt', mtx, delimiter=',', fmt='%.10f', 
               header='Camera Intrinsic Matrix K (fx, 0, cx; 0, fy, cy; 0, 0, 1)')
    print("카메라 내부 행렬 K가 'camera_matrix_K.txt'에 저장되었습니다.")
    
    # 추가로 쉽게 읽을 수 있는 형식으로 저장
    with open('camera_matrix_K_readable.txt', 'w') as f:
        f.write("Camera Intrinsic Matrix K:\n\n")
        f.write("[\n")
        f.write(f"  {mtx[0,0]:.10f}, {mtx[0,1]:.10f}, {mtx[0,2]:.10f}\n")
        f.write(f"  {mtx[1,0]:.10f}, {mtx[1,1]:.10f}, {mtx[1,2]:.10f}\n")
        f.write(f"  {mtx[2,0]:.10f}, {mtx[2,1]:.10f}, {mtx[2,2]:.10f}\n")
        f.write("]\n\n")
        f.write(f"fx = {mtx[0,0]:.10f}\n")
        f.write(f"fy = {mtx[1,1]:.10f}\n")
        f.write(f"cx = {mtx[0,2]:.10f}\n")
        f.write(f"cy = {mtx[1,2]:.10f}\n")
    print("보다 가독성 있는 형식으로 'camera_matrix_K_readable.txt'에도 저장되었습니다.")
else:
    print("코너를 찾을 수 있는 이미지가 없습니다. 캘리브레이션을 수행할 수 없습니다.")

총 53개 이미지 파일을 찾았습니다.
이미지 처리 중: KakaoTalk_Photo_2025-04-01-17-24-15 028.jpeg
  이미지 크기: (1440, 1080, 3)
  체스보드 코너 발견!
이미지 처리 중: KakaoTalk_Photo_2025-04-01-17-24-09 005.jpeg
  이미지 크기: (1440, 1080, 3)
  체스보드 코너 발견!
이미지 처리 중: KakaoTalk_Photo_2025-04-01-17-24-31 002.jpeg
  이미지 크기: (1440, 1080, 3)
  체스보드 코너 발견!
이미지 처리 중: KakaoTalk_Photo_2025-04-01-17-24-14 024.jpeg
  이미지 크기: (1440, 1080, 3)
  체스보드 코너 발견!
이미지 처리 중: KakaoTalk_Photo_2025-04-01-17-24-10 010.jpeg
  이미지 크기: (1440, 1080, 3)
  체스보드 코너 발견!
이미지 처리 중: KakaoTalk_Photo_2025-04-01-17-24-12 019.jpeg
  이미지 크기: (1440, 1080, 3)
  체스보드 코너 발견!
이미지 처리 중: KakaoTalk_Photo_2025-04-01-17-24-13 022.jpeg
  이미지 크기: (1440, 1080, 3)
  체스보드 코너 발견!
이미지 처리 중: KakaoTalk_Photo_2025-04-01-17-24-11 011.jpeg
  이미지 크기: (1440, 1080, 3)
  체스보드 코너 발견!
이미지 처리 중: KakaoTalk_Photo_2025-04-01-17-24-12 018.jpeg
  이미지 크기: (1440, 1080, 3)
  체스보드 코너 발견!
이미지 처리 중: KakaoTalk_Photo_2025-04-01-17-24-14 025.jpeg
  이미지 크기: (1440, 1080, 3)
  체스보드 코너 발견!
이미지 처리 중: KakaoTalk_Photo_202

In [5]:
img = cv.imread('/Users/yangdongjae/Desktop/2025/DGIST/Computer_Vision/Assignment/test_6/data/test5.png')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
 
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

True